In [1]:
!pip install datasets

In [2]:
import pandas as pd
import json
from datasets import load_dataset

In [4]:
# Load the dataset from Hugging Face
dataset = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")

- 데이터셋의 [깃허브](https://github.com/adlnlp/K-MHaS)로부터 확인할 수 있는 각 레이블이 의미하는 바는 다음과 같습니다. 레이블은 0부터 8까지 총 9개의 레이블이 존재합니다.

      class_label:
        names:
          0: origin (출신차별)
          1: physical (외모차별)
          2: politics (정치성향차별)
          3: profanity (혐오욕설)
          4: age (연령차별)
          5: gender (성차별)
          6: race (인종차별)
          7: religion (종교차별)
          8: not_hate_speech (혐오아님)

In [5]:
print('두번째 샘플 출력 :', dataset['text'][0])
print('두번째 샘플의 레이블 출력 :', dataset['label'][0])

두번째 샘플 출력 : "자한당틀딱들.. 악플질 고만해라."
두번째 샘플의 레이블 출력 : [2, 4]


In [6]:
inputs = dataset['text']
labels = dataset['label']

In [7]:
def convert_labels_to_korean(label_lists):
    label_map = {
        0: "출신차별",
        1: "외모차별",
        2: "정치성향차별",
        3: "혐오욕설",
        4: "연령차별",
        5: "성차별",
        6: "인종차별",
        7: "종교차별",
        8: "혐오아님"
    }
    
    return [[label_map[label] for label in label_list] for label_list in label_lists]

In [8]:
kor_labels = convert_labels_to_korean(labels)

In [9]:
inputs[0]

'"자한당틀딱들.. 악플질 고만해라."'

In [10]:
kor_labels[0]

['정치성향차별', '연령차별']

In [11]:
# 이건 추후 라마팩토리에서 추가할 예정
system_prompt = '''당신은 주어진 입력이 어떤 혐오 카테고리에 속하는지를 분류하는 분류기입니다.

1. 아래의 카테고리 중에서만 고르세요.
['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']
2. 주어진 입력은 위 8개의 카테고리 중 여러 개에 속할 수 있습니다.
3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.
4. 입력:이 주어지면 분류 결과: 다음에 파이썬 리스트 형태로 작성하세요.'''

In [12]:
system_prompt

"당신은 주어진 입력이 어떤 혐오 카테고리에 속하는지를 분류하는 분류기입니다.\n\n1. 아래의 카테고리 중에서만 고르세요.\n['출신차별', '외모차별', '정치성향차별', '혐오욕설', '연령차별', '성차별', '인종차별', '종교차별', '혐오아님']\n2. 주어진 입력은 위 8개의 카테고리 중 여러 개에 속할 수 있습니다.\n3. 출력은 반드시 파이썬의 리스트 형태로 생성하십시오.\n4. 입력:이 주어지면 분류 결과: 다음에 파이썬 리스트 형태로 작성하세요."

In [17]:
instruction_lst = []

for input_sample in inputs:
    instruction_lst.append('입력: ' + input_sample)

In [18]:
output_lst = ['분류 결과: ' + str(label) for label in kor_labels]

In [19]:
print(instruction_lst[0])

입력: "자한당틀딱들.. 악플질 고만해라."


In [20]:
print(output_lst[0])

분류 결과: ['정치성향차별', '연령차별']


In [21]:
df = pd.DataFrame({
    'instruction': instruction_lst,
    'output': output_lst,
    'input': ['' for _ in range(len(instruction_lst))] # 라마팩토리 형식을 맞춰주기 위해서 임의로 추가한 열
})

In [22]:
# DataFrame에서 'instruction'과 'output' 열을 딕셔너리 리스트로 변환
data_to_save = df[['instruction', 'input', 'output']].to_dict(orient='records')

# JSON 파일로 저장
with open('hate_speech_detection_data.json', 'w', encoding='utf-8') as f:
    json.dump(data_to_save, f, ensure_ascii=False, indent=4)